<a href="https://colab.research.google.com/github/IshaqKHATTAK/machine-learning-from-scratch-using-python/blob/main/decision_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
pip install anytree

In [ ]:
!pip install dtreeviz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 979 kB/s 
  Created wheel for dtreeviz: filename=dtreeviz-1.3.7-py3-none-any.whl size=68151 sha256=617bc08ae4a8697cde328391c0110d063a10a80fe0dd362ace614e1801d3ab9a
  Stored in directory: /root/.cache/pip/wheels/bf/ba/9f/87c689d8d3c2916793f2dccc57d3dc3b283e0ccf8cb4ca4cad
Successfully built dtreeviz


In [ ]:
df=pd.read_csv('/content/tennis.csv')
df

,Outlook,Temperature,Humidity,Wind,Play Tennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rain,Mild,Normal,Weak,Yes


In [ ]:

#first do some data preprocessing
#steps
#1.  find missing values
#2.  encode categorical data
#3.  spliting dataset
#4.  feature scaling

In [ ]:

#how to convert categorical to numberical
#one hot encoding and label encoder (label encoder is used here)

from sklearn.preprocessing import LabelEncoder

le=LabelEncoder()

for i in df.columns:
  df[i]=le.fit_transform(df[i])

#ID3
1. Calculate entropy for the dataset E(parent).
2. For each node. Calculate entropy for all its categorical values E(feature, child1), E(feature, child2), E(feature, child3) etc.
3. Find the node with highest information gain at a particular level IG(feature) = E(parent) - sum(count_child/count * E(Child)+..).
4. Repeat steps from 1 to 3 till we reach the leaf node and have created our decision tree.

 The above same steps is for C4.5 except we use information ratio instead of informatin gain.

 and for CART we use gini impurity and the rest algorithm is same.

In [ ]:
def entropy(df):
  counts = df.value_counts()
  counts = counts/counts.sum()
  counts = counts * -np.log2(counts)
  return counts

In [ ]:
def info_gain(df,df2):
  #info gain(col) = E(parent) - np.sum(weights of child * E(child))
  parent = entropy(df2)
  parent = parent.sum() # sumup the entropy of each category
  feature_name = df.columns
  info_gain = {}
  for feature in feature_name:
    feature_child_entropy = entropy(df[feature])
    feature_child_counts = df[feature].value_counts()
    feature_child_counts = feature_child_counts / feature_child_counts.sum()
    feature_child_entropy = feature_child_entropy * feature_child_counts
    feature_child_entropy = feature_child_entropy.sum()
    info_gain[feature] = parent - feature_child_entropy
  return max(info_gain, key = info_gain.get) #winner

In [ ]:
y = df['Play Tennis']
x = df.drop(['Play Tennis'],axis = 1)

In [ ]:
def high_info(df,feature,value):
  return df[df['Wind'] == value].reset_index(drop = True)

winner = high_info(df, info_gain(x, y),1)

In [ ]:
winner

,Outlook,Temperature,Humidity,Wind,Play Tennis
0,2,1,0,1,0
1,0,1,0,1,1
2,1,2,0,1,1
3,1,0,1,1,1
4,2,2,0,1,0
5,2,0,1,1,1
6,1,2,1,1,1
7,0,1,1,1,1


In [ ]:
feature = info_gain(x, y)
categories = df[feature].unique()
for value in categories:
  data_value = high_info(x, feature, value)

x = x.drop([feature],axis = 1)


In [ ]:
data_value

,Outlook,Temperature,Humidity,Wind
0,2,1,0,0
1,1,0,1,0
2,0,0,1,0
3,2,2,1,0
4,0,2,0,0
5,1,2,0,0


In [ ]:
df.iloc[0, -1]

0

In [ ]:
a = df['Wind'].unique()

array([1, 0])

In [ ]:
df.drop(['Play Tennis'],axis = 1)

,Outlook,Temperature,Humidity,Wind
0,2,1,0,1
1,2,1,0,0
2,0,1,0,1
3,1,2,0,1
4,1,0,1,1
5,1,0,1,0
6,0,0,1,0
7,2,2,0,1
8,2,0,1,1
9,1,2,1,1


In [ ]:
  values = df['Play Tennis'].unique()
  counts = df['Play Tennis'].value_counts()

In [ ]:
counts = counts/counts.sum()

In [ ]:

counts

1   -1.364694
0   -0.758163
Name: Play Tennis, dtype: float64

In [ ]:
counts.sum()

-0.9402859586706309

In [ ]:
import numpy as np
def find_entropy(df):
  Class = df.keys()[-1]
  values = df[Class].unique()
  entropy = 0
  for value in values:
    prob = df[Class].value_counts()[value]/len(df[Class])
    entropy += -prob * np.log2(prob)
  return np.float(entropy)

In [ ]:
def find_entropy_attribute(df, attribute):
  Class = df.keys()[-1]
  target_values = df[Class].unique()
  attribute_values = df[attribute].unique()
  avg_entropy = 0
  for value in attribute_values:
    entropy = 0
    for value1 in target_values:
      num = len(df[attribute][df[attribute] == value][df[Class] == value1])
      den = len(df[attribute][df[attribute] == value])
      prob = num/den
      entropy += -prob * np.log2(prob + 0.000001)
    avg_entropy += (den/len(df))*entropy
  return np.float(avg_entropy)

In [ ]:
# Find Winner
def find_winner(df):
  IG = {}
  for key in df.keys()[:-1]:
    IG[key]=(find_entropy(df) - find_entropy_attribute(df, key))
  return max(IG, key=IG.get)


In [ ]:
find_winner(df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


'outlook'

In [ ]:
def get_subtable(df, attribute, value):
  return df[df[attribute] == value].reset_index(drop = True)

In [ ]:
def buildtree(df, tree = None):
  node = find_winner(df)
  attvalue = np.unique(df[node])
  Class = df.keys()[-1]
  if tree is None:
    tree = {}
    tree[node] = {}
  for value in attvalue:
    subtable = get_subtable(df,node,value)
    Clvalue, counts = np.unique(subtable[Class], return_counts = True)
    if len(counts) == 1:
      tree[node][value] = Clvalue[0]
    else:
      tree[node][value] = buildtree(subtable)
  return tree

In [ ]:
tre=buildtree(df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


In [ ]:

import pprint
pprint.pprint(tree)

{'outlook': {'overcast': 'yes',
             'rainy': {'windy': {False: 'yes', True: 'no'}},
             'sunny': {'humidity': {'high': 'no', 'normal': 'yes'}}}}


In [ ]:
from anytree import Node, RenderTree
udo = Node("outlook")
overcast = Node("overcast", parent=udo)
joe = Node("play", parent=overcast)
rain = Node("rainy", parent=udo)
sun = Node("sunny", parent=udo)
wind = Node("windy", parent=rain)
no_wind = Node("no wind", parent=wind)
joe1 = Node("play", parent=no_wind)
high_wind = Node("hight wind", parent=wind)
joe2 = Node("don't play", parent=high_wind)
humid = Node("humidity", parent=sun)
nor_humid = Node("normal", parent=humid)
joe3 = Node("play", parent=nor_humid)
high_humid = Node("hight", parent=humid)
joe4 = Node("don't play", parent=high_humid)

In [ ]:
print(udo)
Node('/Udo')
print(joe)
Node('/Udo/Dan/Joe')

Node('/outlook')
Node('/outlook/overcast/play')


Node('//Udo/Dan/Joe')

In [ ]:
for pre, fill, node in RenderTree(udo):
  print("%s%s" % (pre, node.name))

outlook
├── overcast
│   └── play
├── rainy
│   └── windy
│       ├── no wind
│       │   └── play
│       └── hight wind
│           └── don't play
└── sunny
    └── humidity
        ├── normal
        │   └── play
        └── hight
            └── don't play


In [ ]:
from anytree.exporter import DotExporter
DotExporter(udo).to_picture("udo.png")

In [ ]:
DotExporter("udo.png")

regression tree
advantages and disadvantages/problems
ensamble learning

interview questions

1. difference between the classification trees and regression trees
2. how trees work classification and regression trees both.
3. tell me tree algorithm.